In [1]:
from mlxtend.frequent_patterns import apriori, association_rules
import numpy as np
import pandas as pd

In [2]:
product = pd.read_csv(r'C:\Users\angel\Documents\Mago Analytics\PwC\Instacart_MarketBasketAnalysis\data\products.csv')

In [3]:
train = pd.read_csv(r'C:\Users\angel\Documents\Mago Analytics\PwC\Instacart_MarketBasketAnalysis\data\order_products__train.csv')

In [4]:
product = product.set_index('product_id')

train = train.set_index('product_id')

In [5]:
df = train.join(product, how='left', on=train.index)

In [6]:
df = df.reset_index().set_index('order_id')

In [7]:
order = pd.read_csv(r'C:\Users\angel\Documents\Mago Analytics\PwC\Instacart_MarketBasketAnalysis\data\orders.csv')

In [8]:
order['order_dow'].value_counts()

0    600905
1    587478
2    467260
5    453368
6    448761
3    436972
4    426339
Name: order_dow, dtype: int64

In [9]:
order = order.set_index('order_id')

In [10]:
order = order[order['eval_set']=='train']

In [11]:
df = df.join(order, on=df.index, how='left')

In [12]:
df = df.reset_index().set_index(['order_dow', 'order_id'])

In [13]:
df = df.drop(['reordered', 'aisle_id', 'department_id', 'user_id', 'order_number', 'order_hour_of_day', 'days_since_prior_order'], axis=1)

In [14]:
final = df[['product_name']].reset_index()

In [18]:
def mba(df, day):
    # Creating dictionary for days
    days_dict = {0:'Sun', 1:'Mon', 2:'Tues', 3:'Wed', 4:'Thurs', 5:'Fri', 6:'Sat'}
    
    # Adding a unit to the dataframe
    final['units'] = 1
    
    # Creating market basket dataframe with products as columns and transactions as rows
    global df_day
    df_day = pd.pivot_table(data=final[final['order_dow'] == day], 
                            values='units', 
                            index=['order_dow', 'order_id'], 
                            columns='product_name').fillna(0)

    # Creating frequent itemsets
    global freq_items
    freq_items = pd.DataFrame(apriori(df_day.reset_index().drop('order_dow', axis=1).drop('order_id', axis=1), 
                                      min_support=0.01, use_colnames=True, verbose=1, max_len=4))

    global rules
    # Creating metrics: support, confidence, and lift
    rules = pd.DataFrame(association_rules(freq_items, 
                                       metric="confidence", 
                                       min_threshold=0.01).sort_values('confidence', ascending=False))
    rules['day'] = days_dict[day]
    rules.to_csv(('rules'+str(days_dict[day])+'.csv'))

In [19]:
mba(final, 0)

Processing 1086 combinations | Sampling itemset size 3


In [20]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,day
35,(Organic Fuji Apple),(Banana),0.028254,0.171309,0.010450,0.369845,2.158938,0.005609,1.315060,Sun
23,(Honeycrisp Apple),(Banana),0.035063,0.171309,0.012926,0.368640,2.151900,0.006919,1.312549,Sun
43,(Strawberries),(Banana),0.056800,0.171309,0.020353,0.358333,2.091738,0.010623,1.291467,Sun
9,(Organic Hass Avocado),(Bag of Organic Bananas),0.069070,0.129583,0.022902,0.331576,2.558792,0.013952,1.302193,Sun
21,(Broccoli Crown),(Banana),0.032223,0.171309,0.010522,0.326554,1.906227,0.005002,1.230523,Sun
